# Índice

- [8.1 Indexación Jerárquica](#81-indexacion-jerarquica)
  - [Reordenación y clasificación de niveles](#reordenacion-y-clasificacion-de-niveles)
  - [Estadísticas de resumen por nivel](#estadisticas-de-resumen-por-nivel)
  - [Indexación con las columnas de un dataframe](#indexacion-con-las-columnas-de-un-dataframe)

- [8.2 Combinación y Fusión de Datos](#82-combinacion-y-fusion-de-datos)
  - [Uniones de dataframes al estilo de una base de datos](#uniones-de-dataframes-al-estilo-de-una-base-de-datos)
  - [Fusión según el índice](#fusion-segun-el-indice)
  - [Concatenación a lo largo de un eje](#concatenacion-a-lo-largo-de-un-eje)
  - [Combinar datos con superposición](#combinar-datos-con-superposicion)

- [8.3 Remodelación y Transposición](#83-remodelacion-y-transposicion)
  - [Transponer del formato largo al ancho](#transponer-del-formato-largo-al-ancho)
  - [Transponer del formato ancho al largo](#transponer-del-formato-ancho-al-largo)

## 8.1 Indexación Jerárquica
La indexación jerárquica es una característica de pandas que **permite tener varios niveles de índice en un eje** o, dicho de otro modo, proporciona una forma de trabajar con datos de muchas dimensiones en una forma de menor dimensión.

A continuación veremos distintos metodos para entender este concepto.

In [ ]:
#configuracion inicial
import numpy as np # Importo numpy
import pandas as pd # Importo pandas
pd.options.display.max_rows = 20 # maximo 20 filas en la salida
pd.options.display.max_colwidth = 80 # anchura maxima de 80 caracteres de texto a mostrarse
pd.options.display.max_columns = 20 # maximo 20 columnas en la salida
np.random.seed(12345) # define la semilla para numeros aleatorios
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6)) # define la configuracion de los graficos
np.set_printoptions(precision=4, suppress=True) # configuracion de numpy 4 decimales y evita notacion cientifica

Cuando trabajamos con pandas, podemos **representar los datos de dos formas** principales: **Series y DataFrames**.

Una Series es un arreglo unidimensional con un índice que identifica cada valor. Es como una columna de datos con sus etiquetas.

Un DataFrame es una tabla de dos dimensiones con filas y columnas, donde ambos ejes pueden tener índices personalizados o jerárquicos.

Primero veremos ejemplos con Series, porque nos permiten entender el concepto de índice y cómo éste puede ser jerárquico (MultiIndex) para representar varias dimensiones en una sola estructura lineal.

In [ ]:
# creamos una serie con indice jerarquico
gastos = pd.Series(np.random.uniform(size=5),
                 index=[["2025-08-01","2025-08-01", "2025-08-02", "2025-08-03", "2025-08-05"], #Nivel 1 del Indice - Fecha del Gasto
                        ["Supermercado", "Uber", "Colectivo", "Cine", "Farmacia"], #Nivel 2 del Indice - Tipo de Gasto
                        [2500, 4000, 300, 450, 3500], #Nivel 3 del Indice - Monto del Gasto
                        ["Alimentos", "Transporte","Transporte", "Alimentos", "Vestimenta"] #Nivel 4 del Indice - Categoria del Gasto
                        ])
gastos

In [ ]:
gastos.index # vemos como funciona internamente el indice jerarquico

Con objetos jerarquicamente indexados podemos aplicar **indexación parcial**. Esto nos permite seleccionar un subconjunto de datos.

In [ ]:
gastos["2025-08-01":"2025-08-03"] # indexacion parcial por fecha

In [ ]:
gastos.loc[:,:,:,"Transporte"] # idexacion parcial en un nivel interior de la jearquia 

Podemos reordenar estos datos en un DataFrame usando el método **unstack**, que pivotea los niveles del índice en columnas. Por defecto toma el nivel mas interno del indice en columnas.

In [ ]:
gastos.unstack() # reordenamos los datos en un DataFrame usando el metodo unstack. Luego podemos aplicar stack()

Ademas de **series** tambien tenemos otra manera de representar los datos, en **DataFrames**. Con un DataFrame cualquier eje puede tener un indice jerarquico.

Veamoslo con el siguiente ejemplo.

In [ ]:
# creamos un dataframe con ventas de productos por tienda, ciudad y mes
ventas = pd.DataFrame(
    np.array([[200, 150, 300],
              [220, 180, 310],
              [190, 160, 280],
              [210, 170, 320]]),
    index=[["Tienda1", "Tienda1", "Tienda2", "Tienda2"], ["Enero", "Febrero", "Enero", "Febrero"]],
    columns=[["Buenos Aires", "Buenos Aires", "Córdoba"], ["Producto A", "Producto B", "Producto A"]]
)

ventas

Los niveles jerarquicos pueden tener nombres y si los tienen aparecen en el resultado visualizado.

In [ ]:
ventas.index.names = ["Tienda", "Mes"] # ponemos nombres a los niveles del indice de filas
ventas.columns.names = ["Provincia", "Tipo_Producto"] # ponemos nombres a los niveles del indice de columnas
ventas

In [ ]:
ventas.index.nlevels # cantidad de niveles del indice de filas
ventas.columns.nlevels # cantidad de niveles del indice de columnas

Con **indexación de columnas parcial**, al igual que pasa con las series podemos seleccionar grupos de columnas.

In [ ]:
ventas["Buenos Aires"] # selecciono todas las columnas de Buenos Aires

### Reordenación y clasificación de niveles
En ocasiones puede ser necesario modificar el orden de los niveles de un eje o clasificar los datos según los valores de un determinado nivel. El método **swaplevel** toma dos números o nombres de nivel y devuelve un objeto nuevo con los niveles intercambiados (pero, por lo demás, los datos quedan inalterados).

In [ ]:
ventas.swaplevel("Tienda", "Mes") # Tambien se podria indicar la posicion --> ventas.swaplevel(0,1)

**sort_index** clasifica los datos lexicográficamente, utilizando todos los niveles de índice, pero se puede tener la opción de emplear solamente un nivel o un subconjunto de niveles para realizar la clasificación pasando el argumento level.

In [ ]:
ventas.sort_index(level="Mes") # Tambien se podria indicar la posicion --> level=1

### Estadísticas de resumen por nivel
Muchas estadísticas descriptivas y de resumen en objetos DataFrame y Series tienen una opción level, en la que es posible especificar el nivel según el cual se desea agregar en un determinado eje.

In [ ]:
ventas.groupby(level="Tienda").sum()  #suma la cantidad de productos por tienda

In [ ]:
ventas.groupby(level="Provincia", axis="columns").sum()  # total de ventas por tienda y por mes 
# nueva manera de hacerlo en las nuevas versiones de pandas
# ventas.T.groupby(level="Provincia").sum().T

### Indexación con las columnas de un dataframe
En pandas, cada DataFrame tiene un índice de fila, que normalmente es una secuencia de números empezando en 0. Sin embargo, muchas veces queremos que una o varias columnas sirvan como índice, porque esas columnas representan identificadores únicos, categorías importantes o jerarquías de los datos. Esto facilita el acceso a filas específicas y operaciones como agrupamientos o filtrados.
Por otro lado, a veces queremos mover el índice actual a las columnas, por ejemplo si queremos manipularlo o guardarlo como parte de los datos.

In [ ]:
# De manera practica y para seguir con el mismo DataFrame, vamos a resetear los indices en una nueva variable
ventas2 = ventas.reset_index() #El Dataframe ventas ya tenia los indices seteados, por lo que se resetea primero
ventas2

In [ ]:
ventas2.set_index(["Tienda", "Mes"]) #Luego de resetear, se setea

La columnas seleccionadas para setear los indices desaparecen por defecto del DataFrame pero se pueden dejar pasando el argumeto **drop=False** a **set_index**.

In [ ]:
ventas2.set_index(["Tienda", "Mes"], drop = False) # dejo las columnas que setee en el DataFrame

## 8.2 Combinación y Fusión de Datos

Los datos contenidos en objetos pandas se pueden combinar de distintas formas:

🐼 **pandas.merge**: Conecta filas de dataframes según una o varias claves.

🐼 **pandas. concat**: Concatena o apila objetos a lo largo de un eje.

🐼 **combine_first**: Une datos que se solapan para rellenar valores
 ausentes en un objeto con valores de otro.

### Uniones de dataframes al estilo de una base de datos

Las operaciones de fusión o unión combinan conjuntos de datos enlazando filas mediante una o varias filas claves.
La función **pandas.merge** de pandas es el principal punto de entrada para usar estos algoritmos en nuestros datos.
De forma predeterminada, **pandas.merge** realiza una unión "inner";


**Opción**  |  **Comportamiento**

**how="inner"** | Usa solamente las combinaciones de claves observadas en ambas tablas.

**how="left"**  |  Usa todas las combinaciones de claves halladas en la tabla izquierda.

**how="right"**  |  Usa todas las combinaciones de claves halladas en la tabla derecha.

**how="outer"**  |  Usa todas las combinaciones de claves halladas en ambas tablas juntas.






In [ ]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df1
df2
pd.merge(df1, df2, on="key", how="left")

### Fusión según el índice

En caso de que la clave o claves de fusión de un dataframe se encuentren en su indice (etiquetas de fila) se puede pasar left_index=True o right_index=True (o ambos) para indicar que el indice deberia usarse como clave para la unión:

In [ ]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1
right1
pd.merge(left1, right1, left_on="key", right_index=True) # how="outer")

En este ejemplo estariamos utilizando la clave de la derecha para realizar el merge.
Como el indice de right1 es unico, esta union de *muchos a uno*

El objeto DataFrame tiene un metodo de instancia **join** para simplificar la realizacion de uniones según el indice.
Comparado con **pandas.merge**, el metodo join realiza de forma predeterminada una unión a la izquierda según las claves de unión.
Podemos pensar en este metodo como en la union de datos *dentro* del objeto cuyo metodo join fue llamado.
Tambien soporta unir el indice del dataframe pasado en una de las columnas del dataframe


In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
left2
right2
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

In [ ]:
# para reemplazar el merge utilizamos el join
left2.join(right2, how="outer")

# Pasamos la columna por la cual queremos unir el indice
left1.join(right1, on="key")

### Concatenación a lo largo de un eje

La función **pandas.concat()** sirve para unir (concatenar) objetos de pandas, como DataFrame o Series, a lo largo de un eje (filas o columnas).
Es muy útil cuando tienes datos separados y querés combinarlos en un solo objeto.

Supongamos que tenemos tres series sin superposición de indice

In [ ]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [ ]:
s1
s2
s3
pd.concat([s1, s2, s3])

In [ ]:
pd.concat([s1, s2, s3], axis="columns")

Argumentos de la función pandas.concat
Argumento  | Descripción


*   **objs** | lista o diccionario de objetos pandas a concatenar
*   **axis** | Eje a lo largo del cual concatenar; por defecto concatena a lo largo de las filas


*   **join** | Puede ser "inner" u "outer"("outer" por defecto); para intersección o para unión de indices a lo largo de otros ejes.
*   **keys** | valores que se asocian con los objetos que se van a concatenar.
*   **levels** | Índices específicos que se utilizan como índice jerárquico si se pasa keys y/o levels.
*   **names** | Nombres para niveles jerárquicos creados si se pasa keys y/o levels.
*   **verify_integrity** | Busca duplicados en el nuevo eje del objeto concatenado y da un error si los hay.
*   **ignore_index** | No conserva los índices a los largo del axis de concatenación, sino que produce un nuevo índice range(total_index).









In [ ]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], axis="columns")
pd.concat([s1, s4], axis="columns", join="inner")

In [ ]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result
result.unstack()


### Combinar datos con superposición

Si tenemos dos conjuntos de datos con indices que se superponen totalmente o en parte no podemos utilizar las operaciones de fusion o concatenacion.
La funcion **combine_first** podemos decir que *parchea* los datos que faltan en el objeto de llamada con datos del objeto que se pase.

In [ ]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})

df1

In [ ]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df2

In [ ]:
df1.combine_first(df2)

El resultado de **combine_first** con objetos DataFrame tendra la union de todos los nombres de columna.

## 8.3 Remodelación y Transposición
🐼 **pandas.stack**: Cambia o transpone las columnas de los datos por las filas.

🐼 **pandas. unstack**: Cambia o transpone las filas por las columnas.

In [ ]:
# Creamos un DataFrame con índice jerárquico
data = {
    ('Math', 'Score'): [90, 85, 88],
    ('Math', 'Grade'): ['A', 'B', 'B'],
    ('Science', 'Score'): [80, 89, 94],
    ('Science', 'Grade'): ['B', 'A', 'A']
}

df = pd.DataFrame(data, index=['Alice', 'Bob', 'Charlie'])
print("🔹 DataFrame original:")
print(df)

print('\n')
# Usamos stack (convierte columnas en filas)
stacked = df.stack(future_stack=True)
print("🔹 DataFrame con stack:")
print(stacked)

print('\n')
# Usamos unstack (convierte filas en columnas)
unstacked = stacked.unstack()
print("🔹 DataFrame con unstack:")
print(unstacked)

### Transponer del formato largo al ancho

En este formato, los valores individuales se representan con una sola fila de una tabla en lugar de utilizar varios valores por fila.

In [ ]:
# Ejemplo en formato largo (long format)
data = {
    'Fecha': ['2025-01-01', '2025-01-01', '2025-01-02', '2025-01-02'],
    'Ciudad': ['Buenos Aires', 'Córdoba', 'Buenos Aires', 'Córdoba'],
    'Temperatura': [30, 28, 32, 27]
}

df_long = pd.DataFrame(data)
print("🔹 Formato largo:")
print(df_long)

print('\n')
# Convertimos al formato ancho (wide format)
df_wide = df_long.pivot(index='Fecha', columns='Ciudad', values='Temperatura')
print("🔹 Formato ancho:")
print(df_wide)

### Transponer del formato ancho al largo

In [ ]:
# DataFrame en formato ancho
data = {
    'Fecha': ['2025-01-01', '2025-01-02'],
    'Buenos Aires': [30, 32],
    'Córdoba': [28, 27]
}

df_wide = pd.DataFrame(data)
print("🔹 Formato ancho:")
print(df_wide)

# Convertimos al formato largo con melt
df_long = pd.melt(
    df_wide,
    id_vars=['Fecha'],           # Columnas que se mantienen
    var_name='Ciudad',           # Nuevo nombre para las columnas
    value_name='Temperatura'     # Nuevo nombre para los valores
)

print('\n')
print("🔹 Formato largo (con melt):")
print(df_long)